# SchW-BERT-GAN ファインチューニング
Theorytabデータセットを使用
- 事前学習済みのモデルを使用したSchW-BERT-GAN用のモジュールの定義
    - body選択
    - SchwBertGenerator
    - SchwBertDiscriminator
- SchW-BERT-GANのファインチューニングの実施
    - ファインチューニング用関数を作成
    - 事前学習済みのものと比較
- SchW-BERT-GANによる音楽自動生成
    - コードをデータセットからランダムに選んできて生成

WGAN-gpの実装は[caogang/wgan-gp](https://github.com/caogang/wgan-gp/blob/master/gan_mnist.py)が参考になる

In [1]:
import os, time, math, json, random
import datetime
import hickle as hkl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
from pypianoroll import Track, Multitrack
from attrdict import AttrDict
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# カレントディレクトリをリポジトリ直下にして自家製モジュールをimport
while os.getcwd().split('/')[-1] != 'schwbert': os.chdir('..')
print('current dir:', os.getcwd())
from utils import Timer, grid_plot
from bundle import Bundle
from dataloader import TheorytabDataset, TheorytabDataLoader
from model import MusicEmbeddings, MelodyEmbeddings, ChordEmbeddings, ConditionalBertBody
from save_and_load import save_model, load_model
from save_and_load import save_body, load_body
from save_and_load import save_config, load_config

current dir: /root/schwbert


In [2]:
base_dir = "../datasets"
schwbert_dir = os.path.join(base_dir, "schwbert")

input_dir = os.path.join(schwbert_dir, "data", "theorytab")
input_path = os.path.join(input_dir, "original.hkl")

output_base_dir = os.path.join(schwbert_dir, "models")
output_dir = os.path.join(output_base_dir, "theorytab")

for directory in [schwbert_dir, output_base_dir, output_dir]:
    if not os.path.exists(directory):
        os.mkdir(directory)

### データのロード

In [ ]:
with Timer():
     bundle_list = hkl.load(input_path)
dataset = TheorytabDataset(bundle_list)
train_dataset, val_dataset, test_dataset = dataset.split([0.8, 0.17, 0.03], shuffle=True)

print("all bundle size:", len(bundle_list))
print("train size :", len(train_dataset))
print("val   size :", len(val_dataset))
print("test  size :", len(test_dataset))

↓ファインチューニングの節で使うセル

In [ ]:
batch_size = 10
dataloaders_dict = {
    'train': TheorytabDataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': TheorytabDataLoader(val_dataset, batch_size=batch_size, shuffle=True),
    'test': TheorytabDataLoader(test_dataset, batch_size=batch_size/2)
}
print(f"train data size: {len(dataloaders_dict['train'].dataset):<5}, batch num: {len(dataloaders_dict['train']):<5}")
print(f"  val data size: {len(dataloaders_dict['val'].dataset):<5}, batch num: {len(dataloaders_dict['val']):<5}")
print(f" test data size: {len(dataloaders_dict['test'].dataset):<5}, batch num: {len(dataloaders_dict['test']):<5}")

# SchW-BERT-GAN用のモジュールの定義
- SchwBertGenerator*
    - ConditionalBertBody
        - NoiseEmbeddings*: ノイズベクトル→Denseをstep数出力
        - ChordEmbeddings
    - GeneratorHead*: Denseをかけ，(最大出力のところを1にした)?メロディを出力
- SchwBertDiscriminator*
    - ConditionalBertBody
        - MelodyEmebeddings
        - ChordEmbeddings
    - DiscriminatorHead*: (Denseを縦横にかけ)?，本物と偽物を区別するよう2値分類

✳︎ついてるやつが新規作成モジュール

## Generatorの実装

### NoiseEmbeddingsの実装

In [ ]:
class NoiseEmbeddings(MusicEmbeddings):
    def __init__(self, config):
        super(NoiseEmbeddings, self).__init__(config)
        self.noise_size = config.noise_size
        self.input_emb = nn.Linear(self.noise_size, config.hidden_size)
    
    def forward(self, input_tensor):
        # input_tesorはデバイスとサイズ取得用
        
        # ノイズバッチを作成
        
        return super().forward(noise)